# MNIST - Lightning ⚡️ Syft Duet - Data Scientist 🥁

## PART 1: Connect to a Remote Duet Server

As the Data Scientist, you want to perform data science on data that is sitting in the Data Owner's Duet server in their Notebook.

In order to do this, we must run the code that the Data Owner sends us, which importantly includes their Duet Session ID. The code will look like this, importantly with their real Server ID.

```
import syft as sy
duet = sy.duet('xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx')
```

This will create a direct connection from my notebook to the remote Duet server. Once the connection is established all traffic is sent directly between the two nodes.

Paste the code or Server ID that the Data Owner gives you and run it in the cell below. It will return your Client ID which you must send to the Data Owner to enter into Duet so it can pair your notebooks.

In [ ]:
import syft as sy
duet = sy.join_duet(loopback=True)
sy.logger.add(sink="./syft_ds.log")

In [ ]:
duet.store.pandas

## PART 2: Setting up a Model and our Data
The majority of the code below has been adapted closely from the original PyTorch MNIST example which is available in the `original` directory with these notebooks.

The `duet` variable is now your reference to a whole world of remote operations including supported libraries like torch.

Lets take a look at the duet.torch attribute.
```
duet.torch
```

In [ ]:
from pytorch_lightning import Trainer, LightningModule
import torch
import torchvision

Lets create a model just like the one in the MNIST example. We do this in almost the exact same way as in PyTorch. The main difference is we inherit from sy.Module instead of nn.Module and we need to pass in a variable called torch_ref which we will use internally for any calls that would normally be to torch.

In [ ]:
class SyNet(sy.Module):
    def __init__(self, torch_ref):
        super(SyNet, self).__init__(torch_ref=torch_ref)
        self.conv1 = self.torch_ref.nn.Conv2d(1, 32, 3, 1)
        self.conv2 = self.torch_ref.nn.Conv2d(32, 64, 3, 1) 
        self.dropout1 = self.torch_ref.nn.Dropout2d(0.25)
        self.dropout2 = self.torch_ref.nn.Dropout2d(0.5)
        self.fc1 = self.torch_ref.nn.Linear(9216, 128)
        self.fc2 = self.torch_ref.nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.conv2(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.torch_ref.nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = self.torch_ref.flatten(x, 1)
        x = self.fc1(x)
        x = self.torch_ref.nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = self.torch_ref.nn.functional.log_softmax(x, dim=1)
        return output

In [ ]:
from syft.util import get_root_data_path

class Model(LightningModule):
    def __init__(self, local_torch, download_back=False, set_to_local=False):
        super().__init__()
        self.local_model = SyNet(local_torch)
        self.remote_torch = duet.torch
        self.local_torch = local_torch
        self.download_back = download_back
        self.set_to_local = set_to_local
        self.get = self.local_model.get
        self.send = self.local_model.send
        
    def is_remote(self):
        # used to test out everything works locally
        if self.set_to_local:
            return False
        elif not self.trainer.testing:
            return True
        elif self.trainer.evaluation_loop.testing:
            return False
        else:
            return True
    
    @property 
    def torchvision(self):
        return duet.torchvision if self.is_remote() else torchvision
    
    @property 
    def torch(self):
        return self.remote_torch if self.is_remote() else self.local_torch
    
    @property
    def model(self):
        if self.is_remote():
            return self.remote_model
        else:
            if self.download_back:
                return self.get_model()
            else:
                return  self.local_model
    
    def send_model(self):
        self.remote_model = self.local_model.send(duet)
    
    def get_model(self):
        return self.remote_model.get(
            request_block=True,
            reason="test evaluation",
            timeout_secs=5
        )      
    
    def forward(self, x):
        return self.model(x)
    
    def loss(self, output, target):
        return self.torch.nn.functional.nll_loss(output, target)
    
    def training_step(self, batch, batch_idx):
        data_ptr, target_ptr = batch[0], batch[1]
        output = self.forward(data_ptr)
        loss = self.loss(output, target_ptr)
        return loss
    
    def test_step(self, batch, batch_idx):
        data, target = batch[0], batch[1]
        output = self.forward(data)
        loss = self.loss(output, target)
        self.log("test_loss", loss, on_step=True, on_epoch=True, prog_bar=True)
    
    def configure_optimizers(self):
        optimizer = self.torch.optim.SGD(self.model.parameters(), lr=0.1)
        return [optimizer]
    
    def get_transforms(self):
        current_list = duet.python.List if self.is_remote() else list
        transforms = current_list()
        transforms.append(self.torchvision.transforms.ToTensor())
        transforms.append(self.torchvision.transforms.Normalize(0.1307, 0.3081))
        return self.torchvision.transforms.Compose(transforms)
    
    def train_dataloader(self):
        transforms = self.get_transforms()
        train_data_ptr = self.torchvision.datasets.MNIST(str(get_root_data_path()), train=True, download=True, transform=transforms)
        train_loader_ptr = self.torch.utils.data.DataLoader(train_data_ptr, batch_size=64)
        return train_loader_ptr
    
    def test_dataloader(self):
        transforms = self.get_transforms()
        test_data = self.torchvision.datasets.MNIST(str(get_root_data_path()), train=False, download=True, transform=transforms)
        test_loader = self.torch.utils.data.DataLoader(test_data, batch_size=64)
        return test_loader

In [ ]:
model = Model(torch, download_back=False)

In [ ]:
model.send_model()

In [ ]:
# model.get_model()

In [ ]:
trainer = Trainer(
    default_root_dir='./tmp_data',
    max_epochs=2,
    limit_train_batches=2,
    limit_test_batches=2,
    checkpoint_callback=False,
)

In [ ]:
trainer.test(model)

In [ ]:
# TODO solve this
sy.client_cache["duet"] = duet

In [ ]:
trainer.fit(model)

In [ ]:
model.download_back = True
trainer.test(model)

In [ ]:
model.model.get(
    request_block=True,
    reason="test evaluation",
    timeout_secs=5
).save("./duet_lightning_mnist.pt")